<a href="https://colab.research.google.com/github/jaslark/crop-images/blob/main/custom_paddleocr_struct3v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Install PaddlePaddle and PaddleOCR
# 1. Upgrade pip
!pip install --upgrade pip

# 2. Install PaddlePaddle GPU
!pip install paddlepaddle-gpu==3.0.0 -i https://www.paddlepaddle.org.cn/packages/stable/cu118/

# 3. Install PaddleOCR and the missing dependencies causing your error
!pip install paddleocr==2.9.1  # Suggesting 2.9.1 as it is very stable
!pip install langchain langchain-community # <--- THIS FIXES YOUR SPECIFIC ERROR

# 4. Install helpers
!pip install pdf2image PyMuPDF Pillow opencv-python-headless
!apt-get update
!apt-get install -y poppler-utils

In [ ]:
# Cell 2: Verify Installation
import paddle
print(f"PaddlePaddle Version: {paddle.__version__}")
print(f"GPU Available: {paddle.is_compiled_with_cuda()}")
print(f"GPU Device: {paddle.device.get_device()}")

In [ ]:
# Cell 3: Import Libraries and Define Main Class
import os
import json
import zipfile
import shutil
from datetime import datetime
from pathlib import Path
from typing import List, Dict, Union, Optional
import cv2
import numpy as np
from PIL import Image
import fitz  # PyMuPDF

# Create output directories
OUTPUT_DIR = "/content/pp_structure_output"
CROP_DIR = os.path.join(OUTPUT_DIR, "cropped_images")
JSON_DIR = os.path.join(OUTPUT_DIR, "json_results")
ZIP_DIR = os.path.join(OUTPUT_DIR, "zip_files")

for dir_path in [OUTPUT_DIR, CROP_DIR, JSON_DIR, ZIP_DIR]:
    os.makedirs(dir_path, exist_ok=True)

print("Output directories created successfully!")

In [ ]:
# ==========================================
# UPDATED CELL 4 & 5: Robust Processor (PDF Support + Memory Reset)
# ==========================================
import os
import cv2
import json
import zipfile
import numpy as np
import sys
import paddle
from pathlib import Path
from datetime import datetime
from PIL import Image
from pdf2image import convert_from_path

# --- 🔴 CRITICAL FIX: FORCE RESET GLOBAL PROCESSOR ---
# This ensures the old instance (without PDF support) is deleted and replaced.
if '_GLOBAL_PROCESSOR' in globals():
    print("🔄 Resetting Processor Instance to apply updates...")
    _GLOBAL_PROCESSOR = None

class PPStructureV3Processor:
    def __init__(self):
        self.ocr_engine = None

        print("⚙️ Initializing Processor Components...")
        try:
            from paddleocr import PaddleOCR
            print("   • Initializing Standard PaddleOCR Engine...")
            # Initialize OCR engine
            self.ocr_engine = PaddleOCR(
                use_angle_cls=True,
                lang='en',
                use_gpu=True,
                show_log=False
            )
            print("   ✅ PaddleOCR Engine loaded successfully!")
        except Exception as e:
            print(f"   ❌ Critical Error initializing OCR: {e}")

    def process_image_array(self, img_array):
        """Runs OCR on a single numpy image array"""
        if self.ocr_engine:
            try:
                # cls=True enables angle classification
                result = self.ocr_engine.ocr(img_array, cls=True)
                return self._convert_paddleocr_result(result)
            except Exception as e:
                print(f"   ❌ OCR Predict Error: {e}")
        return []

    def _convert_paddleocr_result(self, result):
        """Standardizes PaddleOCR output format"""
        converted = {'text_det_boxes': [], 'rec_texts': [], 'rec_scores': []}

        # Handle empty results
        if not result or result[0] is None:
            return []

        # PaddleOCR structure: [ [ [box], [text, score] ], ... ]
        for line in result[0]:
            converted['text_det_boxes'].append(line[0])
            converted['rec_texts'].append(line[1][0])
            converted['rec_scores'].append(line[1][1])
        return [converted]

    def extract_text_regions(self, result, original_image, filename_prefix, page_num=0):
        """Crops text regions from the image based on OCR detection"""
        cropped_images = []
        text_regions = []

        if not result:
            return [], []

        if isinstance(result, list) and len(result) > 0:
            result = result[0]

        boxes = result.get('text_det_boxes', [])
        texts = result.get('rec_texts', [])
        scores = result.get('rec_scores', [])

        CROP_DIR = "/content/pp_structure_output/cropped_images"
        os.makedirs(CROP_DIR, exist_ok=True)

        for i, box in enumerate(boxes):
            try:
                box = np.array(box).astype(np.int32)

                # Filter noise
                if box.size < 8: continue

                xs = [pt[0] for pt in box]
                ys = [pt[1] for pt in box]

                # Add padding
                pad = 2
                x_min = max(0, int(min(xs)) - pad)
                x_max = min(original_image.shape[1], int(max(xs)) + pad)
                y_min = max(0, int(min(ys)) - pad)
                y_max = min(original_image.shape[0], int(max(ys)) + pad)

                if x_max <= x_min or y_max <= y_min: continue

                crop = original_image[y_min:y_max, x_min:x_max]

                # Naming convention: filename_page_region.png
                crop_name = f"{filename_prefix}_p{page_num+1}_r{i}.png"
                crop_path = os.path.join(CROP_DIR, crop_name)

                # Save crop
                cv2.imwrite(crop_path, crop)

                txt = texts[i] if i < len(texts) else ""
                conf = scores[i] if i < len(scores) else 0.0

                info = {
                    "page": page_num + 1,
                    "region_index": i,
                    "text": txt,
                    "confidence": float(conf),
                    "crop_image": crop_name
                }
                text_regions.append(info)
                cropped_images.append({'path': crop_path, 'filename': crop_name})
            except Exception as e:
                continue

        return cropped_images, text_regions

    def create_output_package(self, base_filename, all_crops, all_regions):
        """Packages everything into a ZIP file"""
        ts = datetime.now().strftime("%Y%m%d_%H%M%S")
        JSON_DIR = "/content/pp_structure_output/json_results"
        ZIP_DIR = "/content/pp_structure_output/zip_files"
        os.makedirs(JSON_DIR, exist_ok=True)
        os.makedirs(ZIP_DIR, exist_ok=True)

        json_name = f"{base_filename}_{ts}.json"
        json_path = os.path.join(JSON_DIR, json_name)
        zip_path = os.path.join(ZIP_DIR, f"{base_filename}_{ts}.zip")

        # Create Master JSON
        output_data = {
            'filename': base_filename,
            'total_pages': max([r['page'] for r in all_regions]) if all_regions else 1,
            'total_regions': len(all_regions),
            'regions': all_regions
        }

        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, indent=2, ensure_ascii=False)

        # Create ZIP
        with zipfile.ZipFile(zip_path, 'w') as zf:
            zf.write(json_path, json_name)
            for crop in all_crops:
                zf.write(crop['path'], f"crops/{crop['filename']}")

        return zip_path

# --- Main Processing Logic ---
def process_document(file_path):
    global _GLOBAL_PROCESSOR

    # Initialize singleton if not present
    if _GLOBAL_PROCESSOR is None:
        _GLOBAL_PROCESSOR = PPStructureV3Processor()

    filename = os.path.basename(file_path)
    base_name = Path(file_path).stem
    ext = Path(file_path).suffix.lower()

    print(f"🚀 Processing: {filename}")

    all_crops = []
    all_regions = []

    # 1. Load Document (PDF or Image)
    images_to_process = [] # List of tuples: (numpy_image, page_index)

    if ext == '.pdf':
        print("   📄 PDF detected. Converting pages to images (this may take a moment)...")
        try:
            # Convert PDF pages to images
            pil_images = convert_from_path(file_path)
            for idx, pil_img in enumerate(pil_images):
                # Convert PIL (RGB) to OpenCV (BGR)
                opencv_img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
                images_to_process.append((opencv_img, idx))
            print(f"   ✅ Converted {len(images_to_process)} pages successfully.")
        except Exception as e:
            print(f"❌ Error converting PDF: {e}")
            return None
    else:
        # Standard Image
        img = cv2.imread(file_path)
        if img is None:
            print("❌ Error: Could not load image file.")
            return None
        images_to_process.append((img, 0))

    # 2. Iterate and Process Each Page
    for img, page_idx in images_to_process:
        print(f"   • Processing Page {page_idx + 1}...")

        # A. Detect Text
        raw_results = _GLOBAL_PROCESSOR.process_image_array(img)

        # B. Extract Crops & Data
        crops, regions = _GLOBAL_PROCESSOR.extract_text_regions(
            raw_results, img, base_name, page_num=page_idx
        )

        all_crops.extend(crops)
        all_regions.extend(regions)

    # 3. Package Results
    if not all_regions:
        print("⚠️ No text found in document.")

    zip_file = _GLOBAL_PROCESSOR.create_output_package(
        base_name, all_crops, all_regions
    )

    return zip_file

print("✓ Robust Processor Loaded (Memory Reset & PDF Support Active)")

In [ ]:
# Cell 6: Upload and Process Interface
from google.colab import files

def upload_and_process():
    """Upload files and process them"""
    print("=" * 60)
    print("       PP-StructureV3 Document Processor")
    print("=" * 60)
    print("\n📤 Please upload your image or PDF file...")
    print("   Supported formats: JPG, PNG, PDF, BMP, TIFF\n")

    # Upload file
    uploaded = files.upload()

    if not uploaded:
        print("❌ No file uploaded!")
        return None

    for filename, content in uploaded.items():
        print(f"\n📄 Processing: {filename}")
        print("-" * 40)

        # Save uploaded file
        input_path = f"/content/{filename}"
        with open(input_path, 'wb') as f:
            f.write(content)

        # Process the document
        try:
            zip_path = process_document(input_path)

            print("\n" + "=" * 60)
            print("✅ Processing Complete!")
            print("=" * 60)

            # Show results summary
            with zipfile.ZipFile(zip_path, 'r') as zf:
                file_list = zf.namelist()
                crop_count = len([f for f in file_list if f.startswith('cropped_images/') and f != 'cropped_images/'])

                print(f"\n📊 Results Summary:")
                print(f"   • Cropped text regions: {crop_count}")
                print(f"   • Output ZIP: {os.path.basename(zip_path)}")

                # Show JSON preview
                for name in file_list:
                    if name.endswith('.json'):
                        with zf.open(name) as jf:
                            json_data = json.load(jf)
                            print(f"\n📝 Detected Text Preview (first 5 regions):")
                            for i, region in enumerate(json_data.get('text_regions', [])[:5]):
                                text = region.get('text', 'N/A')
                                conf = region.get('confidence', 0)
                                print(f"   {i+1}. \"{text[:50]}{'...' if len(text) > 50 else ''}\" (conf: {conf:.2f})")

            # Download the ZIP file
            print(f"\n📥 Downloading: {os.path.basename(zip_path)}")
            files.download(zip_path)

            return zip_path

        except Exception as e:
            print(f"\n❌ Error processing file: {str(e)}")
            import traceback
            traceback.print_exc()
            return None


print("✓ Upload interface ready!")

In [ ]:
# Cell 7: MAIN EXECUTION - Run this cell to process your documents!

print("""
╔══════════════════════════════════════════════════════════════════╗
║          📄 Document Text Detector & Extractor (2025)            ║
╠══════════════════════════════════════════════════════════════════╣
║                                                                  ║
║  🎯 Features:                                                    ║
║     ✓ Text Detection (locate text in images)                     ║
║     ✓ Text Recognition (OCR - read the text)                     ║
║     ✓ Automatic text angle correction                            ║
║     ✓ Cropped images for each text region                        ║
║     ✓ JSON with bounding boxes, text & confidence                ║
║     ✓ Single ZIP download with everything                        ║
║                                                                  ║
║  📁 Supported formats: JPG, PNG, PDF, BMP, TIFF, WEBP            ║
║                                                                  ║
║  📦 Output ZIP contains:                                         ║
║     └── original/          → Your uploaded file                  ║
║     └── cropped_images/    → Individual text region crops        ║
║     └── results/           → JSON with all detection data        ║
║                                                                  ║
╚══════════════════════════════════════════════════════════════════╝
""")

# Process documents
result = upload_and_process()